# Libraries

In [ ]:
!pip install peft bitsandbytes

In [ ]:
import pandas as pd
from datasets import Dataset, load_metric, load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    EarlyStoppingCallback,
    AutoConfig
)

from pathlib import Path
from sklearn.metrics import log_loss
import numpy as np
import json

from peft import get_peft_config, get_peft_model, PeftModel, PeftConfig, LoraConfig, TaskType, LoftQConfig
import bitsandbytes as bnb

from tqdm.auto import tqdm
import random

In [ ]:
from kaggle_secrets import UserSecretsClient
import wandb
user_secrets = UserSecretsClient()
my_secret = user_secrets.get_secret("wandb_api_key") 
wandb.login(key=my_secret)

In [ ]:
%env WANDB_LOG_MODEL="checkpoint"
%env WANDB_PROJECT=LMSYS

# Data & Configs

In [ ]:
def seed_everything(seed):
    import random
    import os
    import numpy as np
    import torch
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(2024)

In [ ]:
max_len = 1024

## Generate label & remove columns

In [ ]:
folder_path = Path('/kaggle/input/lmsys-chatbot-arena')

In [ ]:
train_df = load_dataset('csv', data_files=str(folder_path / 'train.csv'))
# train_df = load_dataset('csv', data_files='external_train.csv')
# train_df = load_dataset('csv', data_files='/kaggle/input/lmsys-additional-33k-labelled-conversations/lmsys-33k-deduplicated.csv')
test_df = load_dataset('csv', data_files=str(folder_path / 'test.csv'))
sample_df = load_dataset('csv', data_files=str(folder_path / 'sample_submission.csv'))

In [ ]:
def get_label(sample):
    sample['label'] = np.argmax([sample['winner_model_a'], sample['winner_model_b'], sample['winner_tie']])
    return sample
train_df = train_df.map(get_label)

In [ ]:
train_df = train_df.select_columns(['prompt', 'response_a', 'response_b', 'label'])
train_df

In [ ]:
dataset = train_df['train'].train_test_split(0.05)
dataset

# Models

In [ ]:
# model_name = "microsoft/deberta-v3-base"
adapther_tokenizer_path = "/kaggle/input/deberta-finetuned/DeBerta-base-5"
model_name = "/kaggle/input/deberta-finetuned/LMSYS/checkpoint-2980"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(adapther_tokenizer_path, model_max_length=max_len)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3,
)

In [ ]:
loftq_config = LoftQConfig(loftq_bits=4)
peft_config = LoraConfig(
#     target_modules=['query_proj', 'value_proj', 'key_proj'],
    use_rslora=True,
    r=16,
    lora_alpha=8,
    lora_dropout=0.1,
    task_type=TaskType.SEQ_CLS,
    init_lora_weights='loftq',
)

model = get_peft_model(model, peft_config)

In [ ]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )
    
print_trainable_parameters(model)

## Tokenize

In [ ]:
sep_token = tokenizer.sep_token_id
cls_token = tokenizer.cls_token_id
tokenizer.add_special_tokens({'pad_token': '<pad>'})
# model.resize_token_embeddings(len(tokenizer))
len_to_each = max_len // 3 - 3

In [ ]:
def tokenize_function(sample):
    prompt =  tokenizer(sample['prompt'], max_length=len_to_each, truncation=True, padding=True).input_ids
    response_a = tokenizer(sample['response_a'], max_length=len_to_each, truncation=True, padding=True).input_ids
    response_b = tokenizer(sample['response_b'], max_length=len_to_each, truncation=True, padding=True).input_ids
    
    sample['input_ids'] = [cls_token] + prompt + [sep_token] + response_a + [sep_token] + response_b
    return sample

In [ ]:
dataset = dataset.map(tokenize_function)

In [ ]:
dataset

In [ ]:
dataset = dataset.select_columns(['input_ids', 'label'])

## Train

In [ ]:
args = TrainingArguments(
    "LMSYS",
    eval_strategy = "steps",
    save_strategy = "steps",
    save_steps=500,
    learning_rate=3e-4,
    fp16=True,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=4,
    optim='adamw_hf',
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    save_total_limit = 2,
    metric_for_best_model='accuracy',
    report_to="wandb",
    run_name="DeBerta-base-train-data",
    eval_steps=500,
    logging_strategy="steps", 
    logging_steps=100,
)

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
    return {
        'eval_log_loss': log_loss(labels, probabilities),
        'eval_accuracy': (np.argmax(logits, axis=1) == labels).mean()
    }

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

In [ ]:
trainer.train()
wandb.finish()

In [ ]:
save_folder = 'DeBerta-base-trained'
model = model.merge_and_unload()
model.save_pretrained(save_folder)
tokenizer.save_pretrained(save_folder)

In [ ]:
trainer.save_model(f"{save_folder}-trainer")